<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Settings

<div class="alert alert-success" role="info">
    <p><b>Note:</b> this data product framework is developed by Naas open source community. You can <a href="https://github.com/sponsors/jupyter-naas" target="_blank">sponsor us</a> if you find it usefull.
</p>
</div>

This notebook allows you to configure and run your data product on your Naas server. *Create an account on [Naas.ai](https://www.naas.ai/free-forever) if you don't already have one.*

Please follow the step below to setup this data product.

## Input

### Install requirements
Install the packages required to run the product.

In [4]:
# !pip install --user --upgrade -r "requirements.txt"

### Import libraries

In [18]:
import naas_data_product
import naas
from naas_drivers import gsheet
import glob
import os
from IPython.display import clear_output

✅ utils file '/home/ftp/abi/utils/data.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_chat_plugin.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_lab.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/openai.ipynb' successfully loaded.


### Setup variables
**Pre-requisite**
- Duplicate our Google Sheets spreadsheet model
- Share your Google Sheets spreadsheet with our service account : 🔗 naas-share@naas-gsheets.iam.gserviceaccount.com

**Inputs**
- `abi_spreadsheet`: Google Sheets spreadsheet URL
- `sheet_entity`: Entity sheet name that stores all your personal data

**Outputs**
- `output_dir_plugin`: Cookie used to authenticate Members and API clients.

In [15]:
# Inputs
abi_spreadsheet = naas.secret.get("ABI_SPREADSHEET")
if abi_spreadsheet is None:
    abi_spreadsheet = input("Enter your Google Sheets URL:")
    naas.secret.add("ABI_SPREADSHEET", abi_spreadsheet)
sheet_entity = "ENTITY"

# Outputs
output_dir = os.path.join("outputs", "entity")
output_dir_plugin = os.path.join("outputs", "plugins")

## Model

### Get data from Google Sheet spreadsheet

In [14]:
df_gsheet = gsheet.connect(abi_spreadsheet).get(sheet_name=sheet_entity).fillna("NA")
print("Rows:", len(df_gsheet))
df_gsheet.head(1)

Rows: 1


,ENTITY,LINKEDIN_URL,PROMPT_ICP,LINKEDIN_LI_AT,LINKEDIN_JSESSIONID,OPENAI_API_KEY
0,Travis Oliphant,https://www.linkedin.com/in/teoliphant/,"I have 2 ideal customer profile, one is a 'Dev...",NA,NA,NA


### Store LinkedIn URL
The LinkedIn URL will be used in content and growth engine.

In [27]:
linkedin_url = df_gsheet.loc[0, "LINKEDIN_URL"]
print("LinkedIn URL:", linkedin_url)
pdump(output_dir, linkedin_url, "linkedin_url")

LinkedIn URL: https://www.linkedin.com/in/teoliphant/


### Store Prompt ICP
The prompt ICP will be used in the growth engine.

In [28]:
prompt_icp = df_gsheet.loc[0, "PROMPT_ICP"]
print("Prompt ICP:", prompt_icp)
pdump(output_dir, prompt_icp, "prompt_icp")

Prompt ICP: I have 2 ideal customer profile, one is a 'Developer' with knowledge of Python. 
The other one is a 'Executive' that will have the budget to buy services to help the developer. 
I will give you the occupation from a profile I get from LinkedIn, you will return stricly and only one of the following values inside the simple quotes based on the best match 'Developer', 'Executive', 'NotICP' or 'NA' if you don't find a plausible match with the first 3 values.
Don't put the results into quotes.


### Add secrets to Naas

In [31]:
for x in ["LINKEDIN_LI_AT", "LINKEDIN_JSESSIONID", "OPENAI_API_KEY"]:
    value = df_gsheet.loc[0, x]
    if value != "NA":
        print(f"Secret '{x}' to be added:")
        naas.secret.add(x, value)

## Output

### List plugin files

In [ ]:
files = glob.glob("*/**/__plugin__.ipynb", recursive=True)
files

### Run plugins

In [ ]:
for file in files:
    print(f"File: {file}")
    %run $file
    clear_output()

### Get plugins URL

In [ ]:
data = []
plugins = glob.glob(f"{output_dir_plugin}/*.json")

for plugin in plugins:
    plugin_url = naas.asset.find(plugin)
    data.append(
        {
            "name": plugin.split("/")[-1].replace(".json", ""),
            "image": "",
            "url": plugin_url,
            "new_chat_url": f"https://naas.ai/chat/use?plugin_url={plugin_url}"
        }
    )
    
data